# Guide to use trained model for predictions
This guide explain how to use a trained model to make predictions of new data set

## 1. Preparing transformation configuration file  

Just like training, predictions using <code>HARDy</code> requires a transformation configuration .yaml file. In this example, only best performing transformation $log(q)$ vs. $d(I(q))/d(q)$ is considered. The configuration file is shown in image below:

<img src="../images/scattering_new_data.png" width=310 align="center" />

The instructions for building transformation configuration file are available at <a href="https://hardy.readthedocs.io/en/latest/examples/How_to_write_Configuration_files.html">How to write configuration files </a>

## 2. Importing required modules

~~~
import hardy
import os
~~~

## 3. Preparing the new data set to be fed into trained model

### Applying transformations to the data

- Defining the location for transformation configuration

``transformation_config_path = './new_data_testing/scattering_tform_config.yaml'``

- Collecting the filenames of new data set having only .csv file extension

``new_data_file_list = [item for item in os.listdir('./new_data_testing/') if item.endswith('.csv')]``

- Loading transformation information from the configuration file

``tform_command_list, tform_command_dict = hardy.arbitrage.import_tform_config(tform_config_path)``

- Defining variables for transforming images
    - ``run_name = 'log_q_der_I'``  
    - ``new_datapath = './new_data_testing/new_data_path/'``
    - ``classes = ['sphere', 'cylinder', 'core-shell', 'ellipsoid']``
    - ``project_name = 'new_data_set'``
    - ``scale = 0.2``
    - ``target_size = (100, 100)``

<i>Please note that the order of <code>classes</code> must be same as used for training of Machine Learning model. Moreover, the <code>scale</code> and <code>target_size</code> must also be the same as used for training.</i>

- Using data wrapper function to generate the rgb images

~~~
hardy.data_wrapper(run_name=run_name, raw_datapath=raw_datapath, tform_command_dict=tform_command_dict, classes='d', scale=0.2)
~~~

<i>Please note that the value for <code>classes</code> used in <code>data_wrapper</code> module is a string rather than classes. This is being done to use same module for different functionalites i-e for training and predictions. Since, the new data set doesn't have the labels, the class <code>'d'</code> is used as assumed class</i>

- The <code>data_rapper</code> will apply the numerical and visual transformations and pickle the data into the <code>new_datapath</code> folder

- To load the transformed data, following code is used:

``transformed_data = hardy.handling.pickled_data_loader(new_datapath, run_name)``

- The data now needs to be converted into iterator acceptable to tensorflow. This can be done by

~~~

new_data_set = hardy.to_catalogue.test_set(image_list=transformed_data, target_size=target_size, classes=classes, color_mode='rgb', iterator_mode='arrays', batch_size=len(new_data_file_list), training=False)

~~~

<i>The argument for <code>training</code> is kept false, to avoid tagging classes in data set. During training, it is kept as true so that model can seek validation of predicted outcomes

The data is now ready to be used for predictions

## 4. Making predictions

- Loading the model

``best_model = hardy.cnn.save_load_model('./model_location/model_name.h5', load=True)``

* Making predictions

``hardy.reporting.model_predictions(best_model, new_test_set, classes, transformed_data)``

This will generate a pandas dataframe outlining the file names, their predicted labels, and probabilities of predictions for each class. It is shown in image below:

<img src="../images/scattering_results_new_data.png" width=310 align="center" />

____